# NHL Player Statistics Script

In [132]:
# Installation Guide for Sportsreference Python Package
## https://sportsreference.readthedocs.io/en/stable/installation.html

# Documentation for Sportsreference Python Package
## https://sportsreference.readthedocs.io/en/stable/nhl.html#

# Importing packages
from datetime import datetime
from sportsreference.nhl.boxscore import Boxscores,Boxscore
import pandas as pd

# Pulling in list of relevant games
games_list = Boxscores(datetime(2020, 8, 1),datetime.today())
print("Step 1 Completed: Pulled in list of relevant games")

# Creating empty lists for data to be populated
game_days = [] # List of days with NHL games
game_ids = [] # List of Game IDs needed to pull game statistics
data = [] # List of collected statistics that will be used to turn into a Dataframe

# Storing in NHL Game Night dates into 'game_days' list
for date in games_list.games:
    game_days.append(date)
print("Step 2 Completed: Stored in NHL Game Night dates into 'game_days' list")

# Storing Game IDs, Home Team Name, Away Team Name and date as a list into 'game_ids' list    
for date in game_days:
    for gameid in games_list.games[date]:
        if gameid['winning_name'] is None:
            print("Removed this bad game data:"+ gameid['boxscore'])
            continue
        else:
            gm_list = []
            gm_list.append(gameid['boxscore'])
            gm_list.append(gameid['home_abbr'])
            gm_list.append(gameid['away_abbr'])
            gm_list.append(date)
            game_ids.append(gm_list)


print("Step 3 Completed: Stored Game IDs into 'game_ids' list")

# Cleaning up 'game_ids' list to remove duplicates        
game_ids = list(set(map(tuple,game_ids )))

# Cleaning up 'game_ids' list to remove blank ids
for ids in game_ids:
    if len(ids[0]) == 0:
        game_ids.remove(ids)

# Sausage Factory
print("Step 4 Starting: Clean up completed...Pulling in player stats")
## Creating a loop that pulls required player statistics for each game_id
for game in game_ids:
    # Home Players
    ## Storing a list of elements containing name, hometeam, game_id, date, goals, assists, points into 'data' list
    home_players = Boxscore(game[0]).home_players
    for hplayer in home_players:
        hm_stat = []
        hm_stat.append(hplayer.name)
        hm_stat.append(game[1]) #hometeam
        hm_stat.append(game[0]) #gameid
        hm_stat.append(game[3]) #date
        hm_stat.append(hplayer.goals)
        hm_stat.append(hplayer.assists)
        hm_stat.append(hplayer.points)
        data.append(hm_stat)
    # Away Players
    ## Storing a list of elements containing name, awayteam, game_id, date, goals, assists, points into 'data' list    
    away_players = Boxscore(game[0]).away_players
    for aplayer in away_players:
        aw_stat = []
        aw_stat.append(aplayer.name)
        aw_stat.append(game[2])
        aw_stat.append(game[0])
        aw_stat.append(game[3])
        aw_stat.append(aplayer.goals)
        aw_stat.append(aplayer.assists)
        aw_stat.append(aplayer.points)
        data.append(aw_stat)

# Transforming the 'data' list into a dataframe and saving it as a CSV
hockey_stats_ts = pd.DataFrame (data, columns = ['Player','Team','Game_ID','Date','Goals','Assists','Points'])
hockey_stats_ts.to_csv('nhl_player_statistics.csv')
print('Detailed NHL player stats have been generated as nhl_player_statistics.csv')

# Grouping up the details stats table above into cumulative player stats and saving it as a CSV
hockey_stats_cum = hockey_stats_ts.groupby(['Player','Team']).sum()
hockey_stats_cum.to_csv('CUM_nhl_player_statistics.csv')
print('Cumulative NHL player stats have been generated as CUM_nhl_player_statistics.csv \n\nStep 4 Completed: CSV Files created...Process Completed')

# Upload Fantasy Teams
print('Step 5 Starting: Uploading Fantasy Teams')
fantasy_team = pd.read_csv("fantasy_teams.csv", encoding='utf-8')
fantasy_team.head()
results = pd.merge(fantasy_team, hockey_stats_cum, how='left',
        left_on='Player', right_on='Player')


# Looking for player names that are not matched
print('Step 6 Starting: Checking For Missing Players')
miss = results.isnull().values.any()
missing = results[results.isna().any(axis=1)]
print('Missing Players Found - Please confirm these are only players who did not play')
print(missing)

# Final Summary Table
print('Here are the current results:')
results_cum = pd.DataFrame(results.groupby(['Fantasy']).sum())
print(results_cum.sort_values(by=['Points'], ascending=False))



Step 1 Completed: Pulled in list of relevant games
Step 2 Completed: Stored in NHL Game Night dates into 'game_days' list
Removed this bad game data:202008060NYR
Removed this bad game data:202008060NYR
Step 3 Completed: Stored Game IDs into 'game_ids' list
Step 4 Starting: Clean up completed...Pulling in player stats
Detailed NHL player stats have been generated as nhl_player_statistics.csv
Cumulative NHL player stats have been generated as CUM_nhl_player_statistics.csv 

Step 4 Completed: CSV Files created...Process Completed
Step 5 Starting: Uploading Fantasy Teams
Step 6 Starting: Checking For Missing Players
Missing Players Found - Please confirm these are only players who did not play
            Player Fantasy  Goals  Assists  Points
100  Nick Schmaltz  Thomas    NaN      NaN     NaN
104   Ryan Dzingel  Thomas    NaN      NaN     NaN
Here are the current results:
         Goals  Assists  Points
Fantasy                        
Klein     14.0     25.0    39.0
Stan      15.0     19.

True
            Player Fantasy  Goals  Assists  Points
100  Nick Schmaltz  Thomas    NaN      NaN     NaN
104   Ryan Dzingel  Thomas    NaN      NaN     NaN
         Goals  Assists  Points
Fantasy                        
Klein     14.0     25.0    39.0
Stan      15.0     19.0    34.0
Daniel    13.0     18.0    31.0
Pottins   15.0     15.0    30.0
Kiyoshi    9.0     18.0    27.0
Mike       7.0     19.0    26.0
Max       10.0     11.0    21.0
Thomas     5.0     14.0    19.0
Steve      7.0      9.0    16.0
Kenny      7.0      5.0    12.0


In [116]:
#hockey_stats_cum.where(hockey_stats_cum['Player'] == 'Mitchell Marner', inplace=True)
df= pd.DataFrame(hockey_stats_cum)#, columns = ['Player','Team','Goals','Assists','Points'])

#hockey_stats_ts.loc[hockey_stats_ts['Player'] == 'Mitch Marner']
hockey_stats_ts.loc[hockey_stats_ts['Player'].str.contains('Dzingel')]


#df.loc[df['Player'] == 'Aaron Ekblad']
#df

,Player,Team,Game_ID,Date,Goals,Assists,Points


In [36]:
hockey

[['Jay Beagle', 'VAN', '202008040VAN', '8-4-2020', 0, 0, 0],
 ['Brock Boeser', 'VAN', '202008040VAN', '8-4-2020', 1, 0, 1],
 ['Alexander Edler', 'VAN', '202008040VAN', '8-4-2020', 0, 2, 2],
 ['Loui Eriksson', 'VAN', '202008040VAN', '8-4-2020', 0, 0, 0],
 ['Oscar Fantenberg', 'VAN', '202008040VAN', '8-4-2020', 0, 0, 0],
 ['Micheal Ferland', 'VAN', '202008040VAN', '8-4-2020', 0, 0, 0],
 ['Bo Horvat', 'VAN', '202008040VAN', '8-4-2020', 1, 1, 2],
 ['Quinn Hughes', 'VAN', '202008040VAN', '8-4-2020', 0, 1, 1],
 ['J.T. Miller', 'VAN', '202008040VAN', '8-4-2020', 1, 1, 2],
 ['Tyler Motte', 'VAN', '202008040VAN', '8-4-2020', 0, 0, 0],
 ['Tyler Myers', 'VAN', '202008040VAN', '8-4-2020', 0, 0, 0],
 ['Tanner Pearson', 'VAN', '202008040VAN', '8-4-2020', 1, 0, 1],
 ['Elias Pettersson', 'VAN', '202008040VAN', '8-4-2020', 0, 1, 1],
 ['Antoine Roussel', 'VAN', '202008040VAN', '8-4-2020', 0, 0, 0],
 ['Troy Stecher', 'VAN', '202008040VAN', '8-4-2020', 0, 0, 0],
 ['Brandon Sutter', 'VAN', '202008040VAN', 

In [47]:
home_players = Boxscore('202008060NYR').home_players
for m in home_players:
    print(m.name)

AttributeError: 'Boxscore' object has no attribute '_home_players'

In [50]:
time = Boxscore('202008060NYR').winning_name
print(time)

TypeError: '>' not supported between instances of 'NoneType' and 'NoneType'

# Older Version

In [ ]:
# Installation Guide for Sportsreference Python Package
## https://sportsreference.readthedocs.io/en/stable/installation.html

# Documentation for Sportsreference Python Package
## https://sportsreference.readthedocs.io/en/stable/nhl.html#

# Importing packages
from sportsreference.nhl.teams import Teams
from sportsreference.nhl.roster import Roster, Player
import pandas as pd



# Creating an empty list that will store player stats; Will be converted into a dataframe
data = []  

# Creating list of NHL teams abbreviations required to pull team roster
teams = Teams()
team_list = []
for team in teams:
    team_list.append(team.abbreviation)
    
# Creating a loop that appends the 'data' list with a list containing player stats 
for team in team_list:
    tm = Roster(team)
    
    # Generating a list of player IDs to pull stats
    players = []
    for player in tm.players:
        players.append(player.player_id)

    # Pulling player statistics into a temporary list and then append it to the 'data' list  
    ## NOTICE: Swap the season to desired season
    for ply in players:
        plyr = Player(ply)
        plyr_st = []
        plyr_st.append(plyr('2019-20').name)
        plyr_st.append(team)
        plyr_st.append(plyr('2019-20').games_played)
        plyr_st.append(plyr('2019-20').points)
        plyr_st.append(plyr('2019-20').goals)
        plyr_st.append(plyr('2019-20').assists)
        data.append(plyr_st)

# Converting the 'data' list to a dataframe called 'stats'
stats = pd.DataFrame (data, columns = ['Player', 'Team','Games Played','Points','Goals','Assists'])

# Exporting dataframe into a CSV
stats.to_csv('nhl_player_statistics.csv')